# Maximum Likelihood Estimation with Bernoulli Distribution

## Import modules

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import os
import sys
import time
import glob

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from IPython import display

import tensorflow as tf

os.environ["CUDA_VISIBLE_DEVICES"]="0"

/home/scpark/anaconda3/envs/ai/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/scpark/anaconda3/envs/ai/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/scpark/anaconda3/envs/ai/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/scpark/anaconda3/envs/ai/lib/python3.7/site-packages

## Setting hyperparameters

In [2]:
# Training Flags (hyperparameter configuration)
max_epochs = 10
batch_size = 128
learning_rate = 1e-1

## Make a toy dataset (Bernoulli distribution)

**Bernoulli distribution**

$X$ is a random variable
$$\Pr(X=1)=p=1-\Pr(X=0)=1-q$$

**Probability mass function**
$$f(k;p)={\begin{cases}p&{\text{if }}k=1,\\q=1-p&{\text{if }}k=0.\end{cases}}$$
or
$$f(k;p)=p^{k}(1-p)^{1-k} \qquad \qquad {\text{for }}k\in \{0,1\}$$

In [3]:
true_p = 0.7
N = 10000
train_data = np.random.binomial(n=1, p=true_p, size=N)
train_data = train_data.astype(np.float32)
train_data = np.expand_dims(train_data, axis=1)

## Set up dataset with `tf.data`

### create input pipeline with `tf.data.Dataset`

In [4]:
# for train
N = len(train_data)
train_dataset = tf.data.Dataset.from_tensor_slices(train_data)
train_dataset = train_dataset.shuffle(buffer_size=N)
train_dataset = train_dataset.batch(batch_size=batch_size, drop_remainder=True)
print(train_dataset)

<BatchDataset shapes: (128, 1), types: tf.float32>


## Create the parameters to learn

**Bernoulli distribution**
$$f(k;p)=p^{k}(1-p)^{1-k} \qquad \qquad {\text{for }}k\in \{0,1\}$$

**Variables**

* `logp`: $\log(p)$

In [5]:
logp = tf.Variable(-1.0) # initial value

In [6]:
def log_pmf(sample, logp):
  epsilon = 1e-7
  return tf.math.log(tf.pow(tf.exp(logp), sample) + epsilon) + \
          tf.math.log(tf.pow(1.-tf.exp(logp), 1.-sample) + epsilon)

## Define the loss functions and the optimizer

In [7]:
optimizer = tf.keras.optimizers.SGD(learning_rate)

## Training

### Define training one step function

In [8]:
# Notice the use of `tf.function`
# This annotation causes the function to be "compiled".
@tf.function
def train_step(data):
  with tf.GradientTape() as tape:
    negative_log_likelihood = -tf.reduce_mean(log_pmf(data, logp))
      
    gradients = tape.gradient(negative_log_likelihood, [logp])
    optimizer.apply_gradients(zip(gradients, [logp]))
  
  return negative_log_likelihood

### Training full steps

In [9]:
print('Start Training.')
num_batches_per_epoch = int(N / batch_size)
global_step = tf.Variable(0, trainable=False)

print("Epochs: {:.2f} global_step: {}  p: {:.3g}".format(
                0., 0, tf.math.exp(logp)))

for epoch in range(max_epochs):
  for step, data in enumerate(train_dataset):
    
    negative_log_likelihood = train_step(data)
    global_step.assign_add(1)
    
    if global_step.numpy() % 50 == 0:
      epochs = epoch + step / float(num_batches_per_epoch)
      #display.clear_output(wait=True)
      print("Epochs: {:.2f} global_step: {} loss: {:.3g}  p: {:.3g}".format(
                epochs, global_step.numpy(), negative_log_likelihood, tf.math.exp(logp)))
      
print('Training Done.')

W0828 23:40:31.436555 139994782705472 deprecation.py:323] From /home/scpark/anaconda3/envs/ai/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1205: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Start Training.
Epochs: 0.00 global_step: 0  p: 0.368
Epochs: 0.63 global_step: 50 loss: 0.692  p: 0.685
Epochs: 1.27 global_step: 100 loss: 0.596  p: 0.695
Epochs: 1.91 global_step: 150 loss: 0.558  p: 0.725
Epochs: 2.55 global_step: 200 loss: 0.604  p: 0.685
Epochs: 3.19 global_step: 250 loss: 0.565  p: 0.704
Epochs: 3.83 global_step: 300 loss: 0.608  p: 0.7
Epochs: 4.47 global_step: 350 loss: 0.665  p: 0.677
Epochs: 5.12 global_step: 400 loss: 0.633  p: 0.676
Epochs: 5.76 global_step: 450 loss: 0.634  p: 0.685
Epochs: 6.40 global_step: 500 loss: 0.646  p: 0.707
Epochs: 7.04 global_step: 550 loss: 0.567  p: 0.699
Epochs: 7.68 global_step: 600 loss: 0.664  p: 0.675
Epochs: 8.32 global_step: 650 loss: 0.646  p: 0.707
Epochs: 8.96 global_step: 700 loss: 0.602  p: 0.695
Epochs: 9.60 global_step: 750 loss: 0.617  p: 0.716
Training Done.


## Print the results

In [10]:
print("Results")
print("p: {:.3g}".format(tf.math.exp(logp)))
print("true p: {:.3g}".format(true_p))

Results
p: 0.703
true p: 0.7
